## Feature Profiling 

- Create feature summary and python feature profiling report

#### Load Packages

In [ ]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
from sqlalchemy import create_engine as ce
from datetime import *
from sklearn import *
from pandas_profiling import ProfileReport

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import warnings
warnings.filterwarnings("ignore")

### Set Parameters

In [ ]:
# List of features that needs to be profiled

features = ["feature_1",
               "feature_2", 
               "feature_3", 
               "feature_4", 
               "feature_5", 
               "feature_6", 
               "feature_7"]

In [ ]:
start_date = "'2023-05-01'"                         # Date is inclusive
end_date = "'2023-08-15'"                           # Date is inclusive

ip = 'xxx.xx.xxx.xxx'                               # IP address
port = xxxx
user = 'username'                                   # username
pass_ai = 'password'                                # password user 
db = 'dbname'                                       # name of schema

main_table = 'table_name'                           # name of table
filter = "isrelevant=1 and on_off = 0"              # filter for table
time_column = 'calltime'                            # column name in table for calltime    

dates_to_filter = "('2022-05-01')"                  # If some dates needs to be removed from data



# unexpected_numeric_values = {'original' : 999999999.0, 'replace' : 999999.0}
# unexpected_string_values = {'original' : '999999999.0', 'replace' : '999999.0'}


# There could be many different ways NAs could be present in dataset. All the following will be converted to numpy.nan
nas_to_replace = ['NA', 'NULL', 'NUL', 'NaN', '[NA]', 'nan', 'NAN', ' ']



In [ ]:
# Limit defined to create a minimized version of feature profile report
features_limit = 10

### Data Fetch

In [ ]:
# data is fetched through following sql query

query = "Select " + time_column + ','+','.join(features)+" from " + main_table + " where " + time_column + " >= " + start_date +" and " + time_column + " <= " + end_date +" and " + filter +" and and " +time_column+" not in "+dates_to_filter+" ;"
print(query)

ai_conn = ce('mysql://'+user+':'+pass_ai+'@'+ip+':'+str(port)+'/'+db)
data = pd.read_sql(query,ai_conn)

print("data fetched successfully : ", data.shape)

In [ ]:
# change column names to lower case
data = data.rename(columns = lambda x: x.lower())

# replace desired values with NAs
for col in data.columns:
    data[col] = data[col].replace(nas_to_replace, np.nan)

#### Features Summary and Trends

In [ ]:
top_values_N = 5                              # Get the top N values for each feature
null_perc_benchmark = 70                      # Features with NULL percentage above this benchmark will be removed from binning
top_1_value_benchmark = 60                    # Features with top 1 value percentage above this benchmark will be removed from binning

special_char = ['$', '&', '%']                 # Any special charcters that needs to be checked for as a value

In [ ]:
# In this section, following stats summary is calculated for each feature
# - unique values - top N values - top N values perc - top 1 value perc - perc of "NA" - perc of nulls - perc of special char

unique_values = []
topN_values = []
topN_values_pct = []
perc_topN_values = []
perc_top1_value = []

perc_stringNA =[]
perc_null=[]
perc_special=[]

data1 = data.copy()
perc_null = data1[data1.columns].isnull().sum()*100/len(data1)
perc_null_df = perc_null.to_frame().reset_index()

dt_types = data1[data1.columns].dtypes
for col in data1.columns:
    series = data1[col].value_counts(normalize=True).to_frame().reset_index()   #fetching all the unique values for each column in descending order
    unique_values.append(len(series[col]))                                      #number of unique values
    topN_values.append(list(series[0:top_values_N]['index']))                   #listing down top N values
    topN_values_pct.append(list(100*round(series[0:top_values_N][col],2)))      #listing down top N values pct
    perc_topN_values.append(100*round(series[0:top_values_N][col].sum(),2))     #Sum of Pct of top N Values
    perc_top1_value.append(100*round(series[0:1][col].sum(),2))                 #pct of top value

    if dt_types[col]=='<M8[ns]':                                                #setting NA/special_char as zero if data type is datetime
        perc_stringNA.append(0)
        perc_special.append(0)
    else:
        perc_stringNA.append(0 if len(series[series['index'].isin(['NA'])][col])==0 
                     else round(100*series[series['index'].isin(['NA'])][col].sum(),2))             #setting as 0 if number of NA are zero else pct of NAs
    
        perc_special.append(0 if len(series[series['index'].isin(special_char)][col])==0 
                        else round(100*series[series['index'].isin(special_char)][col].sum(),2))    #setting as 0 if number of special are zero else pct of special

data_summary = pd.DataFrame({'columns': data1.columns,
                             'unique_values':unique_values,
                             'topN_values':topN_values,
                             'perc_topN_values': perc_topN_values,
                             'perc_top1_value': perc_top1_value,
                             'perc_stringNA': perc_stringNA,
                             'perc_null': round(perc_null_df[0],2),
                             'perc_special': perc_special})


In [ ]:
# save data summary table as excel file on working directory
data_summary.to_excel("./data_summary.xlsx")

In [ ]:
data_summary

### Feature Profiling

In [ ]:
# Python feature profiling report is generated with all sections if number of features is less than the feature_limit
# Else a minimal report is generated
# It is advised to use minimal report if number of features exceed 10-15 bcz of heavy and cluttered interactions and correlations section of report at higher number of features

data = data[features]

if len(data.columns) > features_limit:
#     profile = ProfileReport(data, title = 'Minimal Features Profiling', interactions=None, correlations=None)
    profile = ProfileReport(data, title = 'Minimal Features Profiling', minimal=True)
    profile.to_file('Profiling_Features_minimal_report.html')
else:
    profile = ProfileReport(data, title = 'Features Profiling')
    profile.to_file('Profiling_Features_report.html')

# Report is generated and saved in the working directory